GP model for NOAA AI 2021 Hackathon 'Climate Bench'.



In [ ]:
# Environment setup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from eofs.xarray import Eof
from esem import gp_model

In [ ]:
# Utilities for normalizing the emissions data
min_co2 = 0.
max_co2 = 2400
def normalize_co2(data):
    return data / max_co2

def un_normalize_co2(data):
    return data * max_co2

min_ch4 = 0.
max_ch4 = 0.6
def normalize_ch4(data):
    return data / max_ch4

def un_normalize_ch4(data):
    return data * max_ch4

In [ ]:
#plot inputs
data_path = "C:/Users/maura/OneDrive/Documents/iMIRACLI/hackathon2021/"
from glob import glob

inputs = glob(data_path + "inputs_s*.nc")
SECONDS_IN_YEAR = 60*60*24*365 #s

fig, axes = plt.subplots(2, 2, figsize=(12,12))

for input in inputs:
    label=input.split('_')[1][:-3]
    X = xr.open_dataset(input)
    x = range(2015, 2101)

    weights = np.cos(np.deg2rad(X.latitude))
    
    axes[0, 0].plot(x, X['CO2'].data, label=label)
    axes[0, 0].set_ylabel("Cumulative anthropogenic CO2 \nemissions since 1850 (GtCO2)")
    axes[0, 1].plot(x, X['CH4'].data, label=label)
    axes[0, 1].set_ylabel("Anthropogenic CH4 \nemissions (GtCH4 / year)")
    # FIXME: Not sure where this factor of 1000 comes from...! Maybe the CEDS data is really g/m-2/s?
    axes[1, 0].plot(x, X['SO2'].weighted(weights).sum(['latitude', 'longitude']).data*SECONDS_IN_YEAR*1e-9, label=label)
    axes[1, 0].set_ylabel("Anthropogenic SO2 \nemissions (GtSO2 / year)")
    axes[1, 1].plot(x, X['BC'].weighted(weights).sum(['latitude', 'longitude']).data*SECONDS_IN_YEAR*1e-9, label=label)
    axes[1, 1].set_ylabel("Anthropogenic BC \nemissions (GtBC / year)")

axes[0, 0].set_title('CO2')
axes[0, 1].set_title('CH4')
axes[1, 0].set_title('SO2')
axes[1, 1].set_title('BC')
axes[0, 0].legend()
plt.tight_layout()

In [ ]:
## Set up training data

#adapted from Kai and Theorodore:
def cube_to_vector(data_array):
    "transforms 3 dimensional xarray (time, lat, lon) to 2 dimensional numpy array (time, lat*lon)"
    time_range = data_array["time"].shape[0]
    
    return data_array.values.reshape(time_range,96*144)


def moving_average(a, n) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def create_predictor_data(data_set, n_eofs=5):
    """
    Args:
        data_set (str): name of dataset
        n_eofs (int): number of eofs to create for aerosol variables
    
    """
    X = xr.open_dataset(data_path + "inputs_{}.nc".format(data_set)).compute()
    
    if data_set == "hist-aer":
        X = X.rename_vars({"CO4":"CO2"})
        X = X.sel(time=slice(1850,2100))

    if data_set == "hist-GHG":
        X = X.sel(time=slice(1850,2014))
    
    if "ssp" in data_set or data_set == "hist-aer":
        # Create an EOF solver to do the EOF analysis. Square-root of cosine of
        # latitude weights are applied before the computation of EOFs.
        bc_solver = Eof(X['BC'])

        # Retrieve the leading EOF, expressed as the correlation between the leading
        # PC time series and the input SST anomalies at each grid point, and the
        # leading PC time series itself.
        bc_eofs = bc_solver.eofsAsCorrelation(neofs=n_eofs)
        bc_pcs = bc_solver.pcs(npcs=n_eofs, pcscaling=1)

        # Create an EOF solver to do the EOF analysis. Square-root of cosine of
        # latitude weights are applied before the computation of EOFs.
        so2_solver = Eof(X['SO2'])

        # Retrieve the leading EOF, expressed as the correlation between the leading
        # PC time series and the input SST anomalies at each grid point, and the
        # leading PC time series itself.
        so2_eofs = so2_solver.eofsAsCorrelation(neofs=n_eofs)
        so2_pcs = so2_solver.pcs(npcs=n_eofs, pcscaling=1)

        # Convert the Principle Components of the aerosol emissions (calculated above) in to Pandas DataFrames
        bc_df = bc_pcs.to_dataframe().unstack('mode')
        bc_df.columns = [f"BC_{i}" for i in range(n_eofs)]

        so2_df = so2_pcs.to_dataframe().unstack('mode')
        so2_df.columns = [f"SO2_{i}" for i in range(n_eofs)]
    else:
        # all values are zero, fill up eofs so we have same inputs as for other datasets
        timesteps = X["time"].shape[0]
        zeros = np.zeros(shape=(timesteps, n_eofs))
        bc_df = pd.DataFrame(zeros, columns=[f"BC_{i}" for i in range(n_eofs)],index=X["BC"].coords['time'].data)
        so2_df = pd.DataFrame(zeros, columns=[f"SO2_{i}" for i in range(n_eofs)],index=X["BC"].coords['time'].data)

    # Bring the emissions data back together again and normalise
    inputs = pd.DataFrame({
        "CO2": normalize_co2(X["CO2"].data),
        "CH4": normalize_ch4(X["CH4"].data)
    }, index=X["CO2"].coords['time'].data)

    #5-year running mean of CO2
    co2_1 = normalize_co2(X["CO2"].data)
    co2_5yr=moving_average(co2_1,5)
    nan_array = np.empty((1,4))
    nan_array[:] = np.NaN
    co2_5yr = np.append(nan_array, co2_5yr)
    
    co2_5 = pd.DataFrame({"CO2_5YR": co2_5yr}, index=X["CO2"].coords['time'].data)
    
    # Combine with aerosol EOFs
    inputs=pd.concat([inputs, bc_df, so2_df, co2_5], axis=1)
    
    # replace nan values with 0
    inputs = inputs.replace(np.nan, 0)
    
    return inputs

def create_predicatand_data(data_set):
    Y = xr.open_dataset(data_path + "outputs_{}.nc".format(data_set)).mean("member")
    if data_set == "hist-aer":
        Y = Y.sel(time=slice(1850,2100))
    # Convert the precip values to mm/day
    Y["pr"] *= 86400
    Y["pr90"] *= 86400
    return Y

def create_predicatand_data(data_set):
    Y = xr.open_dataset(data_path + "outputs_{}.nc".format(data_set)).mean("member")
    # Convert the precip values to mm/day
    Y["pr"] *= 86400
    Y["pr90"] *= 86400
        
    return Y

files = ["ssp126","ssp585","historical","1pctCO2","hist-GHG","hist-aer"]

# create training data
X_train = pd.concat([create_predictor_data(file) for file in files])
y_train_tas = np.vstack((cube_to_vector(create_predicatand_data(file)["tas"]) for file in files))
y_train_pr = np.vstack((cube_to_vector(create_predicatand_data(file)["pr"]) for file in files))
y_train_dtf = np.vstack((cube_to_vector(create_predicatand_data(file)["diurnal_temperature_range"]) for file in files))
y_train_pr90 = np.vstack((cube_to_vector(create_predicatand_data(file)["pr90"]) for file in files)) 

files2 = ["ssp370"]

# create test data
X_test = pd.concat([create_predictor_data(file) for file in files2])
test_Y = xr.open_dataset(data_path + 'outputs_ssp370.nc').compute()

tas_truth = test_Y["tas"].mean('member')
pr_truth = test_Y["pr"].mean('member') * 86400
pr90_truth = test_Y["pr90"].mean('member') * 86400
dtr_truth = test_Y["diurnal_temperature_range"].mean('member')



In [ ]:
#pick kernel and train
kernel = ['Linear','RBF']
from esem.data_processors import Whiten, Normalise

tas_gp = gp_model(X_train, y_train_tas,kernel=kernel,kernel_op='mul')
tas_gp.train()

#pr_gp = gp_model(X_train, y_train_pr, kernel=kernel, kernel_op='mul')
#pr_gp.train()

#dtr_gp = gp_model(X_train, y_train_dtr, kernel=kernel, kernel_op='mul')
#dtr_gp.train()

#pr90_gp = gp_model(X_train, y_train_pr90, kernel=kernel, kernel_op='mul')
#pr90_gp.train()


In [ ]:
mod_tas, _ = tas_gp.predict(X_test)

#put output back into pd.DataFrame format for calculating RMSE/plotting
m_tas = np.reshape(mod_tas, [86, 96, 144])
m_tas_data = xr.DataArray(m_tas, dims = tas_truth.dims)

In [ ]:
#test (currently on ssp370 because I don't have the ssp245 output. Test data created above with training data.)

def get_rmse(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    return np.sqrt(((truth-pred)**2).weighted(weights).mean(['lat', 'lon'])).data

print(f"RMSE at 2015: {get_rmse(tas_truth[0], m_tas_data[0])}")
print(f"RMSE at 2050: {get_rmse(tas_truth[35], m_tas_data[35])}")
print(f"RMSE at 2100: {get_rmse(tas_truth[85], m_tas_data[85])}")

RMSE at 2015: 0.44106167100903215
RMSE at 2050: 0.3950025962795656
RMSE at 2100: 2.635233842390099
